In [9]:
import pandas as pd
from sklearn.base import ClassifierMixin
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

algos = (
    LogisticRegression(max_iter=1000),   
    MultinomialNB(), 
    RandomForestClassifier(n_estimators=100),  
    DecisionTreeClassifier(), 
    KNeighborsClassifier(), 
)

In [10]:
X = pd.read_csv("Training Data/Vectorized_X_samples.csv")
X.head()

,abil,abl,abov,absolut,abus,ac,accept,access,account,accus,...,yo,youll,young,youtu,youtub,youtubecom,youv,zero,zombi,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y = pd.read_csv("Training Data/Vectorized_y_samples.csv")
X.shape, y.shape

((37497, 1525), (37497, 1))

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoded_y = encoder.fit_transform(y)

y = pd.Series(encoded_y)
y.head()

0    3
1    3
2    3
3    3
4    3
dtype: int32

In [5]:
class SampleIndices:
    def __init__(self, y):
        self.y = y

    def stratifiedIndices(self, sample_rate:float) -> dict:
        label_indices = []
        for label in self.y.unique():
            label_indices.append(self.y[self.y == label][:int(sample_rate * len(self.y[self.y == label]))].index.tolist())
        label_indices = [val for sublist in label_indices for val in sublist] 
        return label_indices

    def getSamples(self, sample_rate:float=0.7, stratify:bool=True) -> pd.DataFrame:
        """ Sample_rate: float range from 0.1 to 1.0, default=70
            statify: bool, default True """
        if stratify:
            label_indices = self.stratifiedIndices(sample_rate)
            return label_indices
        return self.y.index.tolist()[:int(sample_rate * len(self.y))]
        

sampler = SampleIndices(y=y)
indices = sampler.getSamples(sample_rate=1.0, stratify=True)
print(len(indices))

14997


In [6]:
X_samples = X.iloc[indices]

In [7]:
from abc import ABC, abstractmethod
class HyperParameterTunerAbstract(ABC):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    @abstractmethod
    def gridSearch(self, algo:ClassifierMixin, param_grid:dict):
       pass

    @abstractmethod
    def getBestParams(self, param_grid:dict):
        pass

class HyperParameterTuner(HyperParameterTunerAbstract):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def gridSearch(self, algo:ClassifierMixin, param_grid:dict):
        grid_search = GridSearchCV(estimator=algo, param_grid=param_grid)
        grid_search.fit(self.X, self.y)
        return grid_search

    def getBestParams(self, algo:ClassifierMixin, param_grid:dict):
        grid_search = self.gridSearch(algo, param_grid)
        return grid_search

tuner = HyperParameterTuner(X, y)

In [8]:
# lr_param_grid = {
#     'C': [0.1, 1],
#     'penalty': ['l2']
# }
# lr_grid_search = tuner.getBestParams(LogisticRegression(), param_grid=lr_param_grid)
# lr_grid_search

In [ ]:
param_grids = {
    'LogisticRegression': {
        'C': [0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['lbfgs']
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'MultinomialNB': {
        'alpha': [0.1, 1.0, 10]
    },
    'RandomForestClassifier': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'DecisionTreeClassifier': {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }
}

def get_best_params(model, param_grid, X, y):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X, y)
    return grid_search.best_params_, grid_search.best_score_

models = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'MultinomialNB': MultinomialNB(),
    'RandomForestClassifier': RandomForestClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier()
}

best_params_results = {}

for model_name, model in models.items():
    print(f"Running GridSearchCV for {model_name}")
    best_params, best_score = get_best_params(model, param_grids[model_name], X, y)
    best_params_results[model_name] = {'Best Params': best_params, 'Best Score': best_score}
    print(f"Best Params for {model_name}: {best_params}")
    print(f"Best Score for {model_name}: {best_score}\n")

best_params_results


Running GridSearchCV for LogisticRegression
Best Params for LogisticRegression: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Score for LogisticRegression: 0.46514417227815186

Running GridSearchCV for SVC
